# Data engineering to merge government bridge/tunnel data with existing csv of low clearance bridges

### Bridge Data sourced from  US Dept of Transportation - Federal Highway Administration (https://www.fhwa.dot.gov/bridge/nbi/ascii2018.cfm)
Used this option under delimited files: Download all records. Includes non-highway and routes under bridges zip file (62 mb)

### Tunnel Data sourced from US Dept of Transportation - Federal Highway Administration (https://www.fhwa.dot.gov/bridge/inspection/tunnel/inventory/download.cfm)

Click: Excel

##### Imports ---- dependencies at bottom if needed

In [1]:
import pandas as pd
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

##### Path to local file, different for you

In [3]:
# path to file
bpath = 'C:/Users/Cactuar/Downloads/2018allstatesallrecsdel/2018AllRecordsDelimitedAllStates.txt'
tpath = 'C:/Users/Cactuar/Downloads/2018nti.xlsx'

##### Read-in data, specify encoding for latin characters

In [4]:
bdf = pd.read_csv(bpath, encoding='latin-1')
tdf = pd.read_excel(tpath)

In [5]:
# adjusting options to show all columns
pd.options.display.max_columns = 500

#### Indexable Dictionary of FIPS state codes to their state and abrev

In [6]:
state_dict = {1:('Alabama', 'AL'), 2:('Alaska', 'AK'), 4:('Arizona','AZ'),
              5:('Arkansas','AR'), 6:('California', 'CA'),8:('Colorado', 'CO'),
              9:('Connecticut','CT'),10:('Delaware','DE'),11:('District of Columbia','DC'),
              12:('Flordia','FL'), 13:('Georgia','GA'), 15:('Hawaii','HI'),
              16:('Idaho','ID'), 17:('Illinois','IL'), 18:('Indiana','IN'),
              19:('Iowa','IA'), 20:('Kansas','KS'), 21:('Kentucky','KY'),
              22:('Louisiana','LA'), 23:('Maine','ME'), 24:('Maryland','MD'),
              25:('Massachusetts','MA'), 26:('Michigan','MI'), 27:('Minnesota','MN'),
              28:('Mississippi','MS'), 29:('Missouri','MO'), 30:('Montana','MT'),
              31:('Nebraska','NE'), 32:('Nevada','NV'), 33:('New Hampshire','NH'),
              34:('New Jersey','NJ'), 35:('New Mexico','NM'), 36:('New York','NY'),
              37:('North Carolina','NC'), 38:('North Dakota','ND'), 39:('Ohio','OH'),
              40:('Oklahoma','OK'), 41:('Oregon','OR'), 42:('Pennsylvania','PA'),
              44:('Rhode Island','RI'), 45:('South Carolina','SC'), 46:('South Dakota','SD'),
              47:('Tennessee','TN'), 48:('Texas','TX'), 49:('Utah','UT'),
              50:('Vermont','VT'), 51:('Virginia','VA'), 53:('Washington','WA'),
              54:('West Virginia','WV'), 55:('Wisconsin','WI'), 56:('Wyoming','WY'),
              66:('Guam','GU'),72:('Puerto Rico','PR')}

In [7]:
# # List of all column names, useful to explore for more info
#bcols=bdf.columns.tolist()
##tcols=tdf.columns.tolist()

In [8]:
# #check all values present in column
#bdf["STATE_CODE_001"].value_counts(sort=False)

##### Specify columns to keep

In [9]:
# columns to filter by -> lat, long, state code,clearance , lanes under
bridge_filter= [
    'LAT_016',
    'LONG_017',
    'STATE_CODE_001',
    'MIN_VERT_CLR_010',
    'TRAFFIC_LANES_UND_028B'
]

tunnel_filter = [
    "State Code (I.3)",
    "Tunnel Portal's Latitude (I.13)",
    "Tunnel Portal's Longitude (I.14)",
    "Minimum Vertical Clearance over Tunnel Roadway (G.2)",
]

##### Function to filter bridge dataframe

In [10]:
def filter_b(df, min_clr, max_clr, cols):
    
    # columns to return/filter
    df = df[cols]
    
    
    # create new columns from FIPS dict values
    
    # df["state"] = df['STATE_CODE_001'].apply(lambda x: state_dict[x][0])
    df['st'] = df['STATE_CODE_001'].apply(lambda x: state_dict[x][1])
    df = df.drop(['STATE_CODE_001'], axis=1)
    
    
    # Filter with lanes under bridge > 0
    df = df[df['TRAFFIC_LANES_UND_028B'] != 0]
    
    
    # clearance specifications from *args
    df = df[df['MIN_VERT_CLR_010'] < min_clr]
    df = df[df['MIN_VERT_CLR_010'] > max_clr]
    
    
    # drop nulls
    df = df.dropna()
    
    # convert meters to feet
    df['height'] = df['MIN_VERT_CLR_010'].apply(lambda x: round(x*3.281,1))
    df = df.drop(['MIN_VERT_CLR_010'], axis=1)
    
    
    #convert to str for regex 
    df["LAT_016"] = df["LAT_016"].apply(lambda x: str(int(x)))
    df["LONG_017"] = df["LONG_017"].apply(lambda x:str(int(x)))
    
    
    # make new column of length to use later
    #df["longlen"] = df["LONG_017"].apply(lambda x: len(x))
    
    
    # For arcgis (decimal) format
    eight = lambda x: re.sub(r'(\d\d)(\d\d\d\d\d\d)', r'-\1.\2', x)
    nine = lambda x: re.sub(r'(\d\d\d)(\d\d\d\d\d\d)', r'-\1.\2', x)
    
    
    # # For google maps (degree) input
    # eight = lambda x: re.sub(r'(\d\d)(\d\d)(\d\d)(\d\d)', r'\1 \2 \3.\4W', x)
    # nine = lambda x: re.sub(r'(\d\d\d)(\d\d)(\d\d)(\d\d)',r'\1 \2 \3.\4W', x)
    
    
    # convert lat long to expected format
    
    # # for degree input
    #df["latitude"] = df["LAT_016"].apply(lambda x: re.sub(r'(\d\d)(\d\d)(\d\d)(\d\d)', r'\1 \2 \3.\4N', x))
    
    # for decimal input
    df["latitude"] = df["LAT_016"].apply(lambda x: re.sub(r'(\d\d)(\d\d\d\d\d\d)', r'\1.\2', x))
    df["longitude"] = df["LONG_017"].apply(lambda x:eight(x) if len(x) == 8 else nine(x))
    
    # convert back to int
    df["latitude"] = df["latitude"].apply(lambda x: float(x))
    df["longitude"] = df["longitude"].apply(lambda x: float(x))
    
    
    df = df.drop(['LAT_016', 'LONG_017'], axis=1)
    
    
    # drop remaining unwanted columns
    df = df.drop(['TRAFFIC_LANES_UND_028B'],axis=1)
                                                        
    return df

##### Apply function to dataframe

In [11]:
bdf=filter_b(bdf, 6.1, 2.14, bridge_filter) # 6.1=20 feet, 2.14=7 ft....4.6=15ft

##### Function to filter tunnel dataframe

In [12]:
def filter_t(df, cols):
    
    df = df[cols]
    
    df["st"] = df['State Code (I.3)'].apply(lambda x: state_dict[x][1])
    df = df.drop(['State Code (I.3)'], axis=1)
    
    df['height'] = df['Minimum Vertical Clearance over Tunnel Roadway (G.2)']
    df = df.drop(['Minimum Vertical Clearance over Tunnel Roadway (G.2)'], axis=1)

    df['latitude'] = df["Tunnel Portal's Latitude (I.13)"]
    df = df.drop(["Tunnel Portal's Latitude (I.13)"], axis=1)
    
    df['longitude'] = df["Tunnel Portal's Longitude (I.14)"].apply(lambda x: x*-1)
    df = df.drop(["Tunnel Portal's Longitude (I.14)"], axis=1)
    
    return df

##### Apply function to dataframe

In [13]:
tdf=filter_t(tdf, tunnel_filter)

##### Concatenate the two new dataframes

In [14]:
tbdf = pd.concat([tdf, bdf], axis=0, ignore_index=True)

##### write to csv, your path

In [15]:
# write new csv
path='C:/Users/Cactuar/Downloads/go_low_latlong.csv'
tbdf.to_csv(path,index=False)

##### validate file

In [16]:
df_add = pd.read_csv(path)

In [17]:
df_add.head()

,st,height,latitude,longitude
0,AL,9.7,33.521987,-86.800010
1,AL,12.0,30.690283,-88.040367
2,AL,16.2,30.688789,-88.040133
3,AL,19.2,33.471137,-86.815860
4,AK,14.5,55.343200,-131.645000


In [18]:
len(df_add)

86001

##### path to original csv, make dataframe, drop unwanted column

In [19]:
opath = 'C:/Users/Cactuar/Projects/rv-nav-ds/CSVs/low_clearances.csv'
df_old = pd.read_csv(opath)
df_old = df_old.drop(['Unnamed: 0'], axis=1)

In [20]:
df_old.head()

,uid,latitude,longitude,name,city,st,zip,cat,elev,comments,height
0,100749,34.983726,-86.851996,Low Clearance,Ardmore,AL,35739,LC11,NaN,Height not marked but it is believed to be 11 ...,11.0
1,100750,34.992213,-86.847168,Low Clearance,Ardmore,AL,38449,LC13,NaN,13 - 6 height clearance,13.6
2,100758,34.797197,-86.970228,Low Clearance,Athens,AL,35611,LC10,NaN,10 - 0 height clearance (136),10.0
3,100770,34.018626,-86.085077,Low Clearance,Attalla,AL,35954,LC13,NaN,13 - 7 height clearance,13.7
4,100771,34.021120,-86.087510,Low Clearance,Attalla,AL,35954,LC13,NaN,13 - 8 height clearance,13.8


#### Perform concatenation of the two dataframes

In [21]:
new_df = pd.concat([df_old, df_add], axis=0, ignore_index=True)
new_df.tail()

,cat,city,comments,elev,height,latitude,longitude,name,st,uid,zip
92706,NaN,NaN,NaN,NaN,16.4,17.593925,-66.387481,NaN,PR,NaN,NaN
92707,NaN,NaN,NaN,NaN,16.7,18.242617,-66.015181,NaN,PR,NaN,NaN
92708,NaN,NaN,NaN,NaN,17.6,18.243281,-66.015043,NaN,PR,NaN,NaN
92709,NaN,NaN,NaN,NaN,16.5,18.243000,-66.015900,NaN,PR,NaN,NaN
92710,NaN,NaN,NaN,NaN,17.7,18.244938,-66.013635,NaN,PR,NaN,NaN


##### Write dataframe to new csv

In [22]:
fpath='C:/Users/Cactuar/Downloads/low_clearance_plus_gov.csv'
new_df.to_csv(fpath,index=False)

#### Dependencies

In [23]:
!pip freeze

absl-py==0.8.0
aiohttp==3.5.4
anaconda-client==1.7.2
anaconda-navigator==1.9.7
appdirs==1.4.3
argh==0.26.2
asn1crypto==0.24.0
astor==0.7.1
async-timeout==3.0.1
atomicwrites==1.3.0
attrs==19.1.0
backcall==0.1.0
basilica==0.2.6
beautifulsoup4==4.8.0
black==19.3b0
bleach==3.1.0
blis==0.2.4
boto==2.49.0
boto3==1.9.235
botocore==1.12.235
Bottleneck==1.2.1
browsermob-proxy==0.8.0
cachetools==3.1.1
category-encoders==1.3.0
certifi==2019.6.16
cffi==1.12.3
chardet==3.0.4
Click==7.0
click-plugins==1.1.1
cligj==0.5.0
clyent==1.2.2
colorama==0.4.1
conda==4.7.12
conda-build==3.17.8
conda-package-handling==1.3.11
conda-verify==3.1.1
cryptography==2.7
cssselect==1.1.0
cycler==0.10.0
cymem==2.0.2
decorator==4.4.0
defusedxml==0.6.0
docker-pycreds==0.4.0
docutils==0.15.2
eli5==0.9.0
entrypoints==0.3
enum34==1.1.6
fastai==1.0.58
fastprogress==0.1.21
feedfinder2==0.0.4
feedparser==5.2.1
filelock==3.0.12
Fiona==1.8.9.post2
Flask-SQLAlchemy==2.4.0
funcy==1.13
future==0.17.1
gast==0.2.2
GDAL==2.4.2
gensim==3

In [ ]:
# absl-py==0.8.0
# aiohttp==3.5.4
# anaconda-client==1.7.2
# anaconda-navigator==1.9.7
# appdirs==1.4.3
# argh==0.26.2
# asn1crypto==0.24.0
# astor==0.7.1
# async-timeout==3.0.1
# atomicwrites==1.3.0
# attrs==19.1.0
# backcall==0.1.0
# basilica==0.2.6
# beautifulsoup4==4.8.0
# black==19.3b0
# bleach==3.1.0
# blis==0.2.4
# boto==2.49.0
# boto3==1.9.235
# botocore==1.12.235
# Bottleneck==1.2.1
# browsermob-proxy==0.8.0
# cachetools==3.1.1
# category-encoders==1.3.0
# certifi==2019.6.16
# cffi==1.12.3
# chardet==3.0.4
# Click==7.0
# click-plugins==1.1.1
# cligj==0.5.0
# clyent==1.2.2
# colorama==0.4.1
# conda==4.7.12
# conda-build==3.17.8
# conda-package-handling==1.3.11
# conda-verify==3.1.1
# cryptography==2.7
# cssselect==1.1.0
# cycler==0.10.0
# cymem==2.0.2
# decorator==4.4.0
# defusedxml==0.6.0
# docker-pycreds==0.4.0
# docutils==0.15.2
# eli5==0.9.0
# entrypoints==0.3
# enum34==1.1.6
# fastai==1.0.58
# fastprogress==0.1.21
# feedfinder2==0.0.4
# feedparser==5.2.1
# filelock==3.0.12
# Fiona==1.8.9.post2
# Flask-SQLAlchemy==2.4.0
# funcy==1.13
# future==0.17.1
# gast==0.2.2
# GDAL==2.4.2
# gensim==3.8.0
# geopandas==0.6.1
# gitdb2==2.0.5
# GitPython==3.0.2
# glob2==0.7
# google-api-core==1.14.3
# google-auth==1.6.3
# googleapis-common-protos==1.6.0
# gql==0.1.0
# graphql-core==2.2.1
# graphviz==0.10.1
# grpcio==1.23.0
# gunicorn==19.9.0
# h5py==2.10.0
# idna==2.8
# imageai==2.1.5
# imageio==2.5.0
# importlib-metadata==0.23
# ipykernel==5.1.2
# ipython==7.8.0
# ipython-genutils==0.2.0
# ipywidgets==7.5.1
# jedi==0.15.1
# jellyfish==0.5.6
# jieba3k==0.35.1
# Jinja2==2.10.1
# jmespath==0.9.4
# joblib==0.13.2
# json5==0.8.5
# jsonschema==3.0.2
# jupyter-client==5.3.1
# jupyter-core==4.5.0
# jupyterlab==1.0.2
# jupyterlab-server==1.0.0
# Keras==2.2.5
# Keras-Applications==1.0.8
# Keras-Preprocessing==1.1.0
# kiwisolver==1.1.0
# langdetect==1.0.7
# libarchive-c==2.8
# lxml==4.4.1
# Markdown==3.1.1
# MarkupSafe==1.1.1
# matplotlib==3.1.1
# menuinst==1.4.16
# mistune==0.8.4
# mkl-fft==1.0.14
# mkl-random==1.0.2
# mkl-service==2.3.0
# more-itertools==7.2.0
# mpyq==0.2.5
# multidict==4.5.2
# munch==2.5.0
# murmurhash==1.0.2
# mysql==0.0.2
# mysqlclient==1.4.4
# navigator-updater==0.2.1
# nbconvert==5.5.0
# nbformat==4.4.0
# networkx==2.3
# newspaper3k==0.2.8
# nltk==3.4.5
# notebook==6.0.0
# numexpr==2.7.0
# numpy==1.16.4
# nvidia-ml-py3==7.352.0
# olefile==0.46
# opencv-python==4.1.1.26
# packaging==19.2
# page-objects==1.1.0
# pandas==0.25.1
# pandocfilters==1.4.2
# parso==0.5.1
# pathtools==0.1.2
# patsy==0.5.1
# PDPbox==0.2.0
# pickleshare==0.7.5
# Pillow==6.1.0
# pipenv==2018.11.26
# pkginfo==1.5.0.1
# plac==0.9.6
# pluggy==0.13.0
# portpicker==1.3.1
# preshed==2.0.1
# prometheus-client==0.7.1
# promise==2.2.1
# prompt-toolkit==2.0.9
# protobuf==3.9.1
# psutil==5.6.3
# psycopg2==2.8.3
# py==1.8.0
# pyasn1==0.4.7
# pyasn1-modules==0.2.7
# pycosat==0.6.3
# pycparser==2.19
# pydot==1.4.1
# pygame==1.9.6
# pyglet==1.3.2
# Pygments==2.4.2
# PyHamcrest==1.9.0
# pyLDAvis==2.1.2
# pymongo==3.9.0
# pyOpenSSL==19.0.0
# pyparsing==2.4.2
# pyproj==2.2.1
# pyreadline==2.1
# pyrsistent==0.14.11
# PySC2==2.0.2
# PySocks==1.7.0
# pytest==5.1.3
# python-dateutil==2.8.0
# python-decouple==3.1
# python-dotenv==0.10.3
# pytz==2019.2
# PyWavelets==1.0.3
# pywin32==223
# pywinpty==0.5.5
# PyYAML==5.1.2
# pyzmq==18.1.0
# QtPy==1.9.0
# requests==2.21.0
# requests-file==1.4.3
# rsa==4.0
# Rtree==0.8.3
# ruamel-yaml==0.15.46
# Rx==1.6.1
# s2clientprotocol==4.9.1.74456.0
# s3transfer==0.2.1
# sc2==0.11.1
# scikit-image==0.15.0
# scikit-learn==0.21.2
# scipy==1.3.1
# selenium==3.141.0
# Send2Trash==1.5.0
# sentry-sdk==0.11.2
# Shapely==1.6.4.post2
# shortuuid==0.5.0
# singledispatch==3.4.0.3
# six==1.12.0
# sk-video==1.1.10
# smart-open==1.8.4
# smmap2==2.0.5
# soupsieve==1.9.2
# spacy==2.1.8
# squarify==0.4.3
# srsly==0.1.0
# statsmodels==0.10.1
# subprocess32==3.5.4
# tabulate==0.8.3
# tensorboard==1.13.1
# tensorflow==1.13.1
# tensorflow-estimator==1.13.0
# tensorflow-hub==0.6.0
# termcolor==1.1.0
# terminado==0.8.2
# testpath==0.4.2
# thinc==7.0.8
# tinysegmenter==0.3
# tldextract==2.2.1
# toml==0.10.0
# torch==1.1.0
# torchvision==0.3.0
# tornado==6.0.3
# tqdm==4.32.1
# traitlets==4.3.2
# typing==3.6.4
# urllib3==1.24.3
# virtualenv==16.7.2
# virtualenv-clone==0.5.3
# wandb==0.8.9
# wasabi==0.2.2
# watchdog==0.9.0
# wcwidth==0.1.7
# webencodings==0.5.1
# websocket-client==0.56.0
# Werkzeug==0.15.5
# whichcraft==0.5.2
# widgetsnbextension==3.5.1
# win-inet-pton==1.1.0
# wincertstore==0.2
# xlrd==1.2.0
# yarl==1.3.0
# zipp==0.6.0